<center><div style="font-size : 20px; font-weight : bold; color : darkbue;">PENGANTAR DATA SCIENCE</div>
<div>Eksplorasi Dataset Banking</div>
<hr>
<div>6182001004 | William Surya W - 6182001005 | Kae Zarren L H - 6182001042 | Ester Oka N</div>

In [ ]:
#import library
import pandas as pd;
import numpy as np;

In [ ]:
#load dataset
url = 'https://raw.githubusercontent.com/shrk-sh-ioai/pds-banking-dt/main/bike-dt/day.csv';

forest = pd.read_csv(url, sep=",", encoding='cp1252');

In [ ]:
#data preparation
